# Welcome to the Quantum Parallel Universe

## Initial Setup

### Imports

In [ ]:
import math
from IPython.display import Latex
from qiskit import execute, QuantumCircuit
from qiskit.circuit import Qubit
from qiskit.providers.fake_provider import *
from qiskit.visualization import plot_histogram

### Globals

#### Manually Managed Variables

In [ ]:
# number of qubits: int
N = 16

# shots: int
shots = 4096

# IBMQ Backend
backend = FakeWashingtonV2()

#### Automatically Managed Variables

In [ ]:
# linear GHZ container
linear = {
  'circuit': None,
  'job': None,
  'result': None,
  'time': None,
  'error': { '0': None, '1': None }
}

# logarithmic GHZ container
log = {
  'circuit': None,
  'job': None,
  'result': None,
  'time': None,
  'error': { '0': None, '1': None }
}

# ideal shots per state
isps = shots / 2

# parallel sections
init = [ 0, 1, 2 ]
i = len(init)
k = 1
while len(init) <= N:
  init += [i] * 2**k
  i += 1
  k += 1
s = init[N]

---

## Generate $|\text{GHZ}_N\rangle$ Circuits<sup>1</sup>

### Generate Linear Time Complexity Circuits for $|\text{GHZ}_N\rangle$

In [ ]:
def linear_complexity_GHZ(N: int) -> QuantumCircuit:
  if not isinstance(N, int):
    raise TypeError("Only integer arguments accepted.")
  if N < 1:
    raise ValueError("There must be one or more qubits.")

  c = QuantumCircuit(N)
  for i in range(N):
    c.reset(i)
  c.h(0)
  for i in range(1, N):
    c.cx(i-1, i)
  c.measure_active()
  return c

In [ ]:
linear['circuit'] = linear_complexity_GHZ(N)
linear['circuit'].draw(output='mpl', fold=-1)

### Generate Logaritmic Complexity Circuits for $|\text{GHZ}_{2^m}\rangle$

In [ ]:
def _log_complexity_GHZ(m: int) -> QuantumCircuit:
  if not isinstance(m, int):
    raise TypeError("Only integer arguments accepted.")
  if m < 0:
    raise ValueError("`m` must be at least 0 (evaluated 2^m).")

  if m == 0:
    c = QuantumCircuit([Qubit()])
    c.reset(0)
    c.h(0)
  else:
    c = _log_complexity_GHZ(m - 1)
    for i in range(c.num_qubits):
      c.add_bits([Qubit()])
      new_qubit_index = c.num_qubits - 1
      c.reset(new_qubit_index)
      c.cx(i, new_qubit_index)
  return c

### Generate Logaritmic Complexity Circuits for $|\text{GHZ}_N\rangle$

In [ ]:
def log_complexity_GHZ(N: int) -> QuantumCircuit:
  if not isinstance(N, int):
    raise TypeError("Only an integer argument is accepted.")
  if N < 1:
    raise ValueError("There must be one or more qubits.")

  m = math.ceil(math.log2(N))
  num_qubits_to_erase = 2**m - N
  old_circuit = _log_complexity_GHZ(m=m)
  new_num_qubits = old_circuit.num_qubits - num_qubits_to_erase
  new_circuit = QuantumCircuit(new_num_qubits)
  for gate in old_circuit.data:
    qubits_affected = gate.qubits
    if all(old_circuit.find_bit(qubit).index < new_num_qubits for qubit in qubits_affected):
      new_circuit.append(gate[0], [old_circuit.find_bit(qubit).index for qubit in qubits_affected])
  new_circuit.measure_active()
  return new_circuit


In [ ]:
log['circuit'] = log_complexity_GHZ(N)
log['circuit'].draw(output='mpl', fold=-1)

---

## Quantum Simulation & Results

### Create Simulator Jobs

In [ ]:
linear['job'] = execute(linear['circuit'], backend, shots=shots)
log['job'] = execute(log['circuit'], backend, shots=shots)

### Execution Histograms

#### Linear

In [ ]:
linear['result'] = linear['job'].result()
plot_histogram(linear['result'].get_counts())

#### Logaritmic

In [ ]:
log['result'] = log['job'].result()
plot_histogram(log['result'].get_counts())

---

## Error Analysis

### Linear Error Percentage

##### State $|0\rangle$

In [ ]:
linear['error']['0'] = abs((linear['result'].get_counts()['0' * N] - isps) / isps)
Latex(f"""\\begin{{equation*}}{linear['error']['0'] * 100}\%\\end{{equation*}}""")


##### State $|1\rangle$

In [ ]:
linear['error']['1'] = abs((linear['result'].get_counts()['1' * N] - isps) / isps)
Latex(f"""\\begin{{equation*}}{linear['error']['1'] * 100}\%\\end{{equation*}}""")

### Logarithmic Error Percentage

#### State $|0\rangle$

In [ ]:
log['error']['0'] = abs((log['result'].get_counts()['0' * N] - isps) / isps)
Latex(f"""\\begin{{equation*}}{log['error']['0'] * 100}\%\\end{{equation*}}""")

#### State $|1\rangle$

In [ ]:
log['error']['1'] = abs((log['result'].get_counts()['1' * N] - isps) / isps)
Latex(f"""\\begin{{equation*}}{log['error']['1'] * 100}\%\\end{{equation*}}""")

---

## Speed-Up Analysis

### Run-Times

#### Linear

In [ ]:
linear['time'] = linear['result'].time_taken
Latex(f"""\\begin{{equation*}}{linear['time']}\\space\\text{{seconds}}\\end{{equation*}}""")

#### Log

In [ ]:
log['time'] = log['result'].time_taken
Latex(f"""\\begin{{equation*}}{log['time']}\\space\\text{{seconds}}\\end{{equation*}}""")

### Amdahl's Law

#### Parallel Portion

In [ ]:
S_latency = linear['time'] / log['time']
P = (N * (1 - (1 / S_latency))) / (N - 1)
Latex(f"""\\begin{{equation*}}
    P = \\dfrac{{s\\left(1 - \\dfrac{{1}}{{S_\\text{{latency}}}}\\right)}}{{s - 1}} = \\dfrac{{{s}\\left(1 - \\dfrac{{1}}{{{S_latency}}}\\right)}}{{{s - 1}}} = {P * 100}\%
    \\end{{equation*}}
  """)

#### Sequential Portion

In [ ]:
S_EQ = 1 - P
Latex(f"""\\begin{{equation*}}S_\\text{{EQ}} = 1 - P = {S_EQ * 100}\%\\end{{equation*}}""")

---

## References

1. [arXiv:1807.05572](https://arxiv.org/abs/1807.05572)